Importando base general

In [1]:
# Cargando algunas librerias
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
import datetime

# Importando datos
url = "https://raw.githubusercontent.com/vacunatepe/automatcoviddata/main/datossemana.csv" 
df = pd.read_csv(url,dtype={'UBIGEO': str})
df = df.drop(['Unnamed: 0'], axis=1)
df['DPD'] =  df['DEPARTAMENTO'].map(str) + '-' + df['PROVINCIA'].map(str) + '-' + df['DISTRITO'].map(str)
df.drop(['DEPARTAMENTO', 'PROVINCIA','DISTRITO'], axis=1, inplace=True)
df = df.dropna()
df

,UBIGEO,SEMANA,CONTAGIOS,VACUNADOS,CENTRO_COMERCIAL,TIENDAS_FARMACIAS,PARQUES,PARADEROS,CENTRO_LABORAL,FALLECIDOS,DPD
0,010202,2020-14,1,0.0,0.000000,0.0,-66.714286,-10.428571,-27.714286,0.0,AMAZONAS-BAGUA-ARAMANGO
1,010202,2020-15,3,0.0,0.000000,0.0,-58.571429,-18.571429,-7.571429,0.0,AMAZONAS-BAGUA-ARAMANGO
2,010202,2020-16,1,0.0,0.000000,0.0,-63.142857,-40.428571,-7.000000,0.0,AMAZONAS-BAGUA-ARAMANGO
3,010202,2020-17,11,0.0,0.000000,0.0,-64.285714,-40.428571,0.000000,0.0,AMAZONAS-BAGUA-ARAMANGO
4,010202,2020-18,2,0.0,0.000000,0.0,-61.000000,-38.714286,0.000000,0.0,AMAZONAS-BAGUA-ARAMANGO
...,...,...,...,...,...,...,...,...,...,...,...
40040,250301,2021-05,25,0.0,-22.000000,0.0,4.857143,0.000000,-9.714286,1.0,UCAYALI-PADRE ABAD-PADRE ABAD
40041,250301,2021-06,16,86.0,-23.857143,0.0,1.571429,0.000000,-12.142857,1.0,UCAYALI-PADRE ABAD-PADRE ABAD
40042,250301,2021-07,44,41.0,-14.714286,0.0,2.000000,0.000000,-12.571429,3.0,UCAYALI-PADRE ABAD-PADRE ABAD
40043,250301,2021-08,28,71.0,-7.857143,0.0,-0.428571,0.000000,-9.000000,0.0,UCAYALI-PADRE ABAD-PADRE ABAD


Importando fechas

In [2]:
# Fechas
url2 = "https://raw.githubusercontent.com/vacunatepe/automatcoviddata/main/Fechas.csv" 
fech = pd.read_csv(url2)
fech = fech.drop(['Unnamed: 0'], axis=1)
# Fechas en base general
new_df = pd.merge(df, fech,  how='left', left_on=['SEMANA'], right_on = ['SEMANA'])
new_df['MOVILIDAD'] = new_df[['CENTRO_COMERCIAL', 'TIENDAS_FARMACIAS','PARQUES','PARADEROS','CENTRO_LABORAL']].mean(axis=1)
# Indices
new_df['FECHA'] = pd.to_datetime(new_df['FECHA'], format='%Y-%m-%d')
uniq_idx = new_df.DPD.unique()
new_df.set_index(['DPD','FECHA'], inplace = True)
# Drop NA's
# new_df3['FALLECIDOS'] = new_df3['FALLECIDOS'].fillna(0)
new_df

UBIGEO  ...  MOVILIDAD
DPD                           FECHA               ...           
AMAZONAS-BAGUA-ARAMANGO       2020-04-06  010202  ... -20.971429
                              2020-04-13  010202  ... -16.942857
                              2020-04-20  010202  ... -22.114286
                              2020-04-27  010202  ... -20.942857
                              2020-05-04  010202  ... -19.942857
...                                          ...  ...        ...
UCAYALI-PADRE ABAD-PADRE ABAD 2021-02-01  250301  ...  -5.371429
                              2021-02-08  250301  ...  -6.885714
                              2021-02-15  250301  ...  -5.057143
                              2021-02-22  250301  ...  -3.457143
                              2021-03-01  250301  ...  -1.000000

[24002 rows x 12 columns]

Corriendo un modelo


In [3]:
#Talla de matriz final
k  = uniq_idx.size
df_final = pd.DataFrame(columns=['DPD','Fecha', 'Ubigeo', 'Contagiados','Proyeccion','Fallecidos','Movilidad','Vacunados'], index=uniq_idx)
j = 0
for i in range(k):
  # Define zona
  #-----------------------------------------------------------------------------
  Zona = uniq_idx[i]

  # Genera Modelo
  #-----------------------------------------------------------------------------
  from sklearn import linear_model
  import statsmodels.api as sm
  from statsmodels.sandbox.regression.predstd import wls_prediction_std  
  # Define Variables
  x = np.asanyarray(new_df[['VACUNADOS','FALLECIDOS','CONTAGIOS','MOVILIDAD']].xs(Zona).shift(1))
  y = np.asanyarray(new_df[['CONTAGIOS']].xs(Zona))
  
  try:
   x = np.delete(x, [0,1], 0)
   y = np.delete(y, [0,1], 0)

  # Corre el Modelo
   model = sm.OLS(y, x)
   results = model.fit()

  # Genara variables importantes
   lastdate = new_df.xs(Zona).tail(1).index.item()
   lastobs  = np.asanyarray(new_df['CONTAGIOS'].xs(Zona).tail(1))
   lastx1   = np.asanyarray(new_df['VACUNADOS'].xs(Zona).tail(1))
   lastx2   = np.asanyarray(new_df['FALLECIDOS'].xs(Zona).tail(1))
   lastx3   = np.asanyarray(new_df['CONTAGIOS'].xs(Zona).tail(1))
   lastx4   = np.asanyarray(new_df['MOVILIDAD'].xs(Zona).tail(1))
   lastubigeo = np.asanyarray(new_df['UBIGEO'].xs(Zona).tail(1))
   nextobs = results.params[0]*lastx1 + results.params[1]*lastx2 + results.params[2]*lastx3 + results.params[3]*lastx4

    # Guarda datos
   df_final['DPD'][j] = Zona
   df_final['Fecha'][j]= lastdate
   df_final['Ubigeo'][j] = np.asscalar(lastubigeo)
   df_final['Contagiados'][j] = np.asscalar(lastobs)
   df_final['Proyeccion'][j] = round(np.asscalar(nextobs))
   df_final['Fallecidos'][j] = np.asscalar(lastx2)
   df_final['Movilidad'][j] = np.asscalar(lastx4)
   df_final['Vacunados'][j] = np.asscalar(lastx1)
  except:
        pass
  j = j+1
df_final  

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:42: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: np.asscalar(a) is deprecated since NumPy v1.16, use a.item() instead
/usr/loca

,DPD,Fecha,Ubigeo,Contagiados,Proyeccion,Fallecidos,Movilidad,Vacunados
AMAZONAS-BAGUA-ARAMANGO,AMAZONAS-BAGUA-ARAMANGO,2021-03-01 00:00:00,010202,1,14,0,-13.2,5
AMAZONAS-BAGUA-BAGUA,AMAZONAS-BAGUA-BAGUA,2021-03-01 00:00:00,010201,13,9,0,-13.2,199
AMAZONAS-BAGUA-COPALLIN,AMAZONAS-BAGUA-COPALLIN,2021-02-01 00:00:00,010203,3,3,0,-16.4286,0
AMAZONAS-BAGUA-EL PARCO,AMAZONAS-BAGUA-EL PARCO,2021-02-08 00:00:00,010204,1,4,0,-15.8571,0
AMAZONAS-BAGUA-IMAZA,AMAZONAS-BAGUA-IMAZA,2021-03-01 00:00:00,010205,2,26,0,-13.2,14
...,...,...,...,...,...,...,...,...
UCAYALI-PADRE ABAD-ALEXANDER VON HUMBOLDT,UCAYALI-PADRE ABAD-ALEXANDER VON HUMBOLDT,2021-02-15 00:00:00,250305,1,1,0,-5.05714,0
UCAYALI-PADRE ABAD-CURIMANA,UCAYALI-PADRE ABAD-CURIMANA,2021-02-22 00:00:00,250303,1,2,0,-3.45714,36
UCAYALI-PADRE ABAD-IRAZOLA,UCAYALI-PADRE ABAD-IRAZOLA,2021-03-01 00:00:00,250302,2,1,0,-1,0
UCAYALI-PADRE ABAD-NESHUYA,UCAYALI-PADRE ABAD-NESHUYA,2021-03-01 00:00:00,250304,3,2,0,-1,0


In [4]:
for i in uniq_idx:
#  try:
  if df_final['Proyeccion'][i] < 0:
    df_final['Proyeccion'][i] = df_final['Contagiados'][i]
#  except:
#        pass   

df_final.to_csv('Proyeccion.csv')
df_final

,DPD,Fecha,Ubigeo,Contagiados,Proyeccion,Fallecidos,Movilidad,Vacunados
AMAZONAS-BAGUA-ARAMANGO,AMAZONAS-BAGUA-ARAMANGO,2021-03-01 00:00:00,010202,1,14,0,-13.2,5
AMAZONAS-BAGUA-BAGUA,AMAZONAS-BAGUA-BAGUA,2021-03-01 00:00:00,010201,13,9,0,-13.2,199
AMAZONAS-BAGUA-COPALLIN,AMAZONAS-BAGUA-COPALLIN,2021-02-01 00:00:00,010203,3,3,0,-16.4286,0
AMAZONAS-BAGUA-EL PARCO,AMAZONAS-BAGUA-EL PARCO,2021-02-08 00:00:00,010204,1,4,0,-15.8571,0
AMAZONAS-BAGUA-IMAZA,AMAZONAS-BAGUA-IMAZA,2021-03-01 00:00:00,010205,2,26,0,-13.2,14
...,...,...,...,...,...,...,...,...
UCAYALI-PADRE ABAD-ALEXANDER VON HUMBOLDT,UCAYALI-PADRE ABAD-ALEXANDER VON HUMBOLDT,2021-02-15 00:00:00,250305,1,1,0,-5.05714,0
UCAYALI-PADRE ABAD-CURIMANA,UCAYALI-PADRE ABAD-CURIMANA,2021-02-22 00:00:00,250303,1,2,0,-3.45714,36
UCAYALI-PADRE ABAD-IRAZOLA,UCAYALI-PADRE ABAD-IRAZOLA,2021-03-01 00:00:00,250302,2,1,0,-1,0
UCAYALI-PADRE ABAD-NESHUYA,UCAYALI-PADRE ABAD-NESHUYA,2021-03-01 00:00:00,250304,3,2,0,-1,0
